In [ ]:
import pandas as pd
from tqdm import tqdm
from time import sleep
import matplotlib.pyplot as plt
from collections import Counter
import re
import seaborn as sns
import numpy as np

In [ ]:
doctors = pd.read_excel("../data/doctors.xlsx", sheet_name='main')
nurses = pd.read_excel("../data/nurses.xlsx", sheet_name='main')

In [ ]:
print("NaN values in 'doctors':", doctors.isna().sum())
print("NaN values in 'nurses':", nurses.isna().sum())

In [ ]:
# nurses.rename(columns={"НЕГОСУДАРСТВЕННАЯ ФОРМА СОБСТВЕННОСТИ": "private", "ВСЕГО (по всем формам собственности)": "total", "Государственная собственность": "public", "ВСЕГО": "total"}, inplace=True)
# doctors.rename(columns={"НЕГОСУДАРСТВЕННАЯ ФОРМА СОБСТВЕННОСТИ": "private", "ВСЕГО (по всем формам собственности)": "total", "Государственная собственность": "public", "ВСЕГО": "total"}, inplace=True)

In [ ]:
doctors.head()

In [ ]:
# Filter out aggregate regions
filtered = doctors[~doctors['region'].str.contains('Российская Федерация|федеральный округ')]

# Get total doctors for each region in 2016 and 2023
total_2016 = filtered[filtered['year'] == 2016].set_index('region')['total']
total_2023 = filtered[filtered['year'] == 2023].set_index('region')['total']

# Only consider regions present in both years
common_regions = total_2016.index.intersection(total_2023.index)

# Count regions where total in 2023 < total in 2016
shrunk_count = (total_2023[common_regions] < total_2016[common_regions]).sum()
print("Number of regions where total doctors shrank in 2023 vs 2016:", shrunk_count)

In [ ]:
# Exclude aggregate regions
filtered = doctors[~doctors['region'].str.contains('Российская Федерация|федеральный округ')]

# Pivot to have years as columns and regions as rows
pivot = filtered.pivot(index='region', columns='year', values='total')

def max_consecutive_decline(row):
    declines = (row.diff() < 0).astype(int)
    max_streak = streak = 0
    for val in declines[1:]:
        if val:
            streak += 1
            max_streak = max(max_streak, streak)
        else:
            streak = 0
    return max_streak

consecutive_decline = pivot.apply(max_consecutive_decline, axis=1)
count_3plus = (consecutive_decline >= 3).sum()
print("Number of regions with three or more years of consecutive decline:", count_3plus)

In [ ]:
# Exclude aggregate regions
filtered = doctors[~doctors['region'].str.contains('Российская Федерация|федеральный округ')]

# Pivot to have years as columns and regions as rows
pivot = filtered.pivot(index='region', columns='year', values='total')

# For each year (except the first), count regions where total decreased vs previous year
loss_counts = {}
years = sorted(pivot.columns)
for i in range(1, len(years)):
    prev, curr = years[i-1], years[i]
    loss_counts[curr] = (pivot[curr] < pivot[prev]).sum()

# Create a table
loss_table = pd.DataFrame(list(loss_counts.items()), columns=['year', 'regions_lost_doctors'])
print("Regions where total doctors decreased compared to previous year:")
print(loss_table)


In [ ]:
pivot.head()

In [ ]:
for year in sorted(doctors['year'].unique()):
    df_year = doctors[(doctors['year'] == year) & (~doctors['region'].str.contains('Российская Федерация')) & (~doctors['region'].str.contains('федеральный округ'))]
    min_row = df_year.loc[df_year['total'].idxmin()]
    max_row = df_year.loc[df_year['total'].idxmax()]
    print(f"{year}:")
    print(f"Lowest: {min_row['region']} - {min_row['total']}")
    print(f"Highest: {max_row['region']} - {max_row['total']}")

In [ ]:
doctors.columns

In [ ]:
# Calculate the percentage change in total doctors for each region between the earliest and latest year
region_changes = doctors[~doctors['region'].str.contains('Российская Федерация|федеральный округ')].copy()
first_year = region_changes['year'].min()
last_year = region_changes['year'].max()

first = region_changes[region_changes['year'] == first_year].set_index('region')['total']
last = region_changes[region_changes['year'] == last_year].set_index('region')['total']

# Only keep regions present in both years
common_regions = first.index.intersection(last.index)
pct_change = (last[common_regions] - first[common_regions]).sort_values()

print("Regions with largest decrease:")
print(pct_change.head(10))

print("\nRegions with largest increase:")
print(pct_change.tail(10))

In [ ]:
doctors['region'].unique()

In [ ]:
# Exclude 'Тюменская область' but keep the specified subregions
filtered_regions_tyumen = [
    r for r in filtered_regions
    if 'тюменская область' not in r.lower().strip() or
       'ханты-мансийский автономный округ' in r.lower() or
       'ямало-ненецкий автономный округ' in r.lower() or
       'тюменская область (кроме ханты-мансийского автономного округа-югры и ямало-ненецкого автономного округа)' in r.lower()
]

In [ ]:
# Exclude specified regions and those mentioning 'российская федерация' or 'федеральный округ'
filtered_regions = [
    r for r in common_regions
    if 'российская федерация' not in r.lower()
    and 'федеральный округ' not in r.lower()
    and all(ex.strip() != r.strip() for ex in exclude)
]

gained = (last[filtered_regions] > first[filtered_regions]).sum()
lost = (last[filtered_regions] < first[filtered_regions]).sum()
unchanged = (last[filtered_regions] == first[filtered_regions]).sum()

print(f"Regions that gained doctors: {gained}")
print(f"Regions that lost doctors: {lost}")
print(f"Regions with no change: {unchanged}")


In [ ]:
filtered_regions

In [ ]:
doctors.loc[(doctors['region'] == 'Чукотский автономный округ') & (doctors['year'] == 2023), 'total']

### Nurses

In [ ]:
nurses.dtypes

In [ ]:
for year in sorted(nurses['year'].unique()):
    df_year = nurses[(nurses['year'] == year) & (~nurses['region'].str.contains('Российская Федерация|федеральный округ'))]
    min_row = df_year.loc[df_year['total'].idxmin()]
    max_row = df_year.loc[df_year['total'].idxmax()]
    print(f"{year}:")
    print(f"Lowest: {min_row['region']} - {min_row['total']}")
    print(f"Highest: {max_row['region']} - {max_row['total']}")

In [ ]:
# Calculate the percentage change in total nurses for each region between the earliest and latest year
region_changes = nurses[~nurses['region'].str.contains('Российская Федерация|федеральный округ')].copy()
first_year = region_changes['year'].min()
last_year = region_changes['year'].max()

first = region_changes[region_changes['year'] == first_year].set_index('region')['total']
last = region_changes[region_changes['year'] == last_year].set_index('region')['total']

# Only keep regions present in both years
common_regions = first.index.intersection(last.index)
pct_change = ((last[common_regions] - first[common_regions]) / first[common_regions] * 100).sort_values()

print("Regions with largest percentage decrease:")
print(pct_change.head(10))

print("\nRegions with largest percentage increase:")
print(pct_change.tail(10))

In [ ]:
# Calculate the change in total nurses for each region between the earliest and latest year
region_changes = nurses[~nurses['region'].str.contains('Российская Федерация|федеральный округ')].copy()
first_year = region_changes['year'].min()
last_year = region_changes['year'].max()

first = region_changes[region_changes['year'] == first_year].set_index('region')['total']
last = region_changes[region_changes['year'] == last_year].set_index('region')['total']

# Only keep regions present in both years
common_regions = first.index.intersection(last.index)
pct_change = (last[common_regions] - first[common_regions]).sort_values()

print("Regions with largest percentage decrease:")
print(pct_change.head(10))

print("\nRegions with largest percentage increase:")
print(pct_change.tail(10))